In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf


# Import and clean df

In [2]:
# Import cleaned VAERS dataset
vaers_df = pd.read_csv('VAERSDATA.csv', low_memory=False)
vaers_df.head()

,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,...,NaN,NaN,NaN,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,...,NaN,NaN,NaN,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",...,23.1,NaN,NaN,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",...,NaN,NaN,NaN,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",...,23.1,NaN,NaN,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))


In [3]:
# Check vaers_df column types
vaers_df.dtypes

Unnamed: 0           int64
VAERS_ID             int64
RECVDATE            object
STATE               object
AGE_YRS            float64
CAGE_YR            float64
CAGE_MO            float64
SEX                 object
RPT_DATE            object
SYMPTOM_TEXT        object
DIED                object
DATEDIED            object
L_THREAT            object
ER_VISIT            object
HOSPITAL            object
HOSPDAYS           float64
X_STAY              object
DISABLE             object
RECOVD              object
VAX_DATE            object
ONSET_DATE          object
NUMDAYS            float64
LAB_DATA            object
V_ADMINBY           object
V_FUNDBY            object
OTHER_MEDS          object
CUR_ILL             object
HISTORY             object
PRIOR_VAX           object
SPLTTYPE            object
FORM_VERS            int64
TODAYS_DATE         object
BIRTH_DEFECT        object
OFC_VISIT           object
ER_ED_VISIT         object
ALLERGIES           object
SYMPTOM1            object
S

In [4]:
# change dates(objects)columns to datetime
vaers_df['RECVDATE'] = pd.to_datetime(vaers_df['RECVDATE'])
vaers_df['RPT_DATE'] = pd.to_datetime(vaers_df['RPT_DATE'])
vaers_df['DATEDIED'] = pd.to_datetime(vaers_df['DATEDIED'])
vaers_df['VAX_DATE'] = pd.to_datetime(vaers_df['VAX_DATE'])
vaers_df['ONSET_DATE'] = pd.to_datetime(vaers_df['ONSET_DATE'])
vaers_df['TODAYS_DATE'] = pd.to_datetime(vaers_df['TODAYS_DATE'])

vaers_df.dtypes

Unnamed: 0                  int64
VAERS_ID                    int64
RECVDATE           datetime64[ns]
STATE                      object
AGE_YRS                   float64
CAGE_YR                   float64
CAGE_MO                   float64
SEX                        object
RPT_DATE           datetime64[ns]
SYMPTOM_TEXT               object
DIED                       object
DATEDIED           datetime64[ns]
L_THREAT                   object
ER_VISIT                   object
HOSPITAL                   object
HOSPDAYS                  float64
X_STAY                     object
DISABLE                    object
RECOVD                     object
VAX_DATE           datetime64[ns]
ONSET_DATE         datetime64[ns]
NUMDAYS                   float64
LAB_DATA                   object
V_ADMINBY                  object
V_FUNDBY                   object
OTHER_MEDS                 object
CUR_ILL                    object
HISTORY                    object
PRIOR_VAX                  object
SPLTTYPE      

In [5]:
# Categorical variable list
vaers_cat = vaers_df.dtypes[vaers_df.dtypes == 'object'].index.tolist()
vaers_cat

['STATE',
 'SEX',
 'SYMPTOM_TEXT',
 'DIED',
 'L_THREAT',
 'ER_VISIT',
 'HOSPITAL',
 'X_STAY',
 'DISABLE',
 'RECOVD',
 'LAB_DATA',
 'V_ADMINBY',
 'V_FUNDBY',
 'OTHER_MEDS',
 'CUR_ILL',
 'HISTORY',
 'PRIOR_VAX',
 'SPLTTYPE',
 'BIRTH_DEFECT',
 'OFC_VISIT',
 'ER_ED_VISIT',
 'ALLERGIES',
 'SYMPTOM1',
 'SYMPTOM2',
 'SYMPTOM3',
 'SYMPTOM4',
 'SYMPTOM5',
 'VAX_TYPE',
 'VAX_MANU',
 'VAX_LOT',
 'VAX_DOSE_SERIES',
 'VAX_ROUTE',
 'VAX_SITE',
 'VAX_NAME']

In [6]:
# Unique values in each column
vaers_df[vaers_cat].nunique()

STATE                 60
SEX                    3
SYMPTOM_TEXT       27195
DIED                   1
L_THREAT               1
ER_VISIT               1
HOSPITAL               1
X_STAY                 1
DISABLE                1
RECOVD                 3
LAB_DATA            7726
V_ADMINBY              9
V_FUNDBY               4
OTHER_MEDS         11816
CUR_ILL             3518
HISTORY             9515
PRIOR_VAX           1033
SPLTTYPE            5895
BIRTH_DEFECT           1
OFC_VISIT              1
ER_ED_VISIT            1
ALLERGIES           6245
SYMPTOM1            2301
SYMPTOM2            2220
SYMPTOM3            2017
SYMPTOM4            1833
SYMPTOM5            1617
VAX_TYPE              40
VAX_MANU              14
VAX_LOT             1887
VAX_DOSE_SERIES        8
VAX_ROUTE              7
VAX_SITE               9
VAX_NAME              63
dtype: int64

In [8]:
# LAB DATA value counts
counts = vaers_df.LAB_DATA.value_counts()
counts

None                                                                                                                                                                                                                                                       2684
none                                                                                                                                                                                                                                                       1557
no                                                                                                                                                                                                                                                          222
NONE                                                                                                                                                                                                                                    

In [12]:
# Drop LAB_DATA, OTHER MEDS,  - unless we use NLP, the column can't be encoded
vaers_df.drop(columns = ['LAB_DATA', 'OTHER_MEDS'])

,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,...,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,0,916600,2021-01-01,TX,33.0,33.0,NaN,F,NaT,Right side of epiglottis swelled up and hinder...,...,NaN,NaN,NaN,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,1,916601,2021-01-01,CA,73.0,73.0,NaN,F,NaT,Approximately 30 min post vaccination administ...,...,NaN,NaN,NaN,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,2,916602,2021-01-01,WA,23.0,23.0,NaN,F,NaT,"About 15 minutes after receiving the vaccine, ...",...,23.1,NaN,NaN,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,3,916603,2021-01-01,WA,58.0,58.0,NaN,F,NaT,"extreme fatigue, dizziness,. could not lift my...",...,NaN,NaN,NaN,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,4,916604,2021-01-01,TX,47.0,47.0,NaN,F,NaT,"Injection site swelling, redness, warm to the ...",...,23.1,NaN,NaN,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40256,40256,1092595,2021-03-11,TX,70.0,70.0,NaN,F,NaT,A high risk employee received 1st vaccine dose...,...,NaN,NaN,NaN,COVID19,MODERNA,025J20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
40257,40257,1092651,2021-03-11,CA,77.0,77.0,NaN,M,NaT,DEATH,...,NaN,NaN,NaN,COVID19,MODERNA,0124214,1,IM,LA,COVID19 (COVID19 (MODERNA))
40258,40258,1092653,2021-03-11,CA,88.0,88.0,NaN,F,NaT,Patient received the vaccine on the evening of...,...,NaN,NaN,NaN,COVID19,MODERNA,unknown,2,SYR,AR,COVID19 (COVID19 (MODERNA))
40259,40259,1092737,2021-03-11,ME,69.0,69.0,NaN,M,NaT,"Sudden death. Alone at home, found on floor 4 ...",...,NaN,NaN,NaN,COVID19,MODERNA,NaN,1,UN,AR,COVID19 (COVID19 (MODERNA))


In [11]:
# OTHER MEDS value counts
counts = vaers_df.OTHER_MEDS.value_counts()
counts

None                                                                                                            1866
none                                                                                                             875
unknown                                                                                                          360
Unknown                                                                                                          359
No                                                                                                               131
                                                                                                                ... 
LISINOPRIL; LEVOTHYROXIN; PRAVASTATIN; VITAMIN D; VITAMIN K                                                        1
furosemide, tacrolimus, myfortic                                                                                   1
Type 1 diabetic- taking Humalog (carb adjusted before meals) and

In [15]:
# CUR_ILL value counts
counts = vaers_df.CUR_ILL.value_counts()
counts

None                                                    6322
none                                                    3455
No                                                       654
no                                                       522
NONE                                                     365
                                                        ... 
Only severe acid reflux                                    1
No recent illness or vaccination within prior month.       1
heart related issues, stents per daughter                  1
none per patient                                           1
miscarriage                                                1
Name: CUR_ILL, Length: 3518, dtype: int64

In [16]:
# HISTORY value counts
counts = vaers_df.HISTORY.value_counts()
counts

None                                                                                              3153
none                                                                                              1525
Comments: List of non-encoded Patient Relevant History: Patient Other Relevant History 1: None     676
Medical History/Concurrent Conditions: No adverse event (No reported medical history)              375
No                                                                                                 339
                                                                                                  ... 
TB positive skin reaction. No history of infection                                                   1
eczema  asthma occasional heartburn                                                                  1
diabetes 2                                                                                           1
Diabetes Osteoarthritis Dyshidrosis                                      

In [17]:
# PRIOR_VAX value counts
counts = vaers_df.PRIOR_VAX.value_counts()
counts

Flu                                                                                      18
unknown                                                                                  13
Flu Vaccine                                                                              10
COVID19 (Moderna) on 1/26/2021                                                            9
Dizziness, mild chest pain, mid spine pain, back of neck pain, headaches, body aches,     8
                                                                                         ..
adverse reaction to flu vaccine (fever, rash, hives)                                      1
Per vaccination form, no information available                                            1
Age 20? With yellow fever vaccine.  High fever 24 hours.                                  1
Patient reported mother told her she needed to get half doses of vaccines as a child      1
itch at shot sight                                                              

In [18]:
#SPLTTYPE value counts
counts = vaers_df.SPLTTYPE.value_counts()
counts

USMODERNATX, INC.MOD20210    1291
vsafe                         514
VSAFE                          73
USMODERNATX, INC.MOD20200      49
USGLAXOSMITHKLINEUS2020GS      29
                             ... 
USPFIZER INC2021093210          1
USPFIZER INC2021122444          1
USPFIZER INC2021156890          1
USPFIZER INC2021013567          1
USPFIZER INC2020521395          1
Name: SPLTTYPE, Length: 5895, dtype: int64

In [19]:
#ALLERGIES value counts
counts = vaers_df.ALLERGIES.value_counts()
counts

None                                                                         3369
none                                                                         1619
NKA                                                                           804
NKDA                                                                          769
No                                                                            401
                                                                             ... 
Dust bites, dogs, pollen, trees                                                 1
Morphine and Reglan                                                             1
Banana, avocado, honeydew melon, cantaloupe, pineapple, yellow watermelon       1
Naproxen, Lyrica                                                                1
Fish, ibuprofen, PCN, Sulfa, Tetracycline, Turkey                               1
Name: ALLERGIES, Length: 6245, dtype: int64

In [20]:
#SYMPTOM1 value counts
counts = vaers_df.SYMPTOM1.value_counts()
counts

Chills                         2358
Arthralgia                     1516
Headache                       1197
Injection site erythema        1176
Dizziness                      1175
                               ... 
Hypomagnesaemia                   1
Nasal flaring                     1
Alcohol interaction               1
Hyperdynamic left ventricle       1
Varicella                         1
Name: SYMPTOM1, Length: 2301, dtype: int64

In [21]:
#SYMPTOM2 value counts
counts = vaers_df.SYMPTOM2.value_counts()
counts

Headache                        1683
Fatigue                         1301
Chills                          1118
Dizziness                        835
Pyrexia                          825
                                ... 
Myelocyte count                    1
Peripheral vascular disorder       1
Chlamydia test negative            1
Libido decreased                   1
Partial seizures                   1
Name: SYMPTOM2, Length: 2220, dtype: int64

In [22]:
#VAX_LOT value counts
counts = vaers_df.VAX_LOT.value_counts()
counts

039K20A      1375
011J20A      1260
EK9231       1132
EK5730       1072
EH9899       1047
             ... 
YF41408         1
015M02A         1
025J2D-2A       1
E9262           1
EH4899          1
Name: VAX_LOT, Length: 1887, dtype: int64

In [29]:
# Drop LAB_DATA, OTHER MEDS, CUR_ILL, HISTORY, PRIOR_VAX, ALLERGIES - unless we use NLP, those columns can't be hot encoded
vaers_clean = vaers_df.drop(columns = ['LAB_DATA', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'ALLERGIES'])

In [30]:
# re-count Unique values in each column in new cleaned df
vaers_clean.nunique()

Unnamed: 0         40261
VAERS_ID           27955
RECVDATE              70
STATE                 60
AGE_YRS              118
CAGE_YR              105
CAGE_MO                7
SEX                    3
RPT_DATE              20
SYMPTOM_TEXT       27195
DIED                   1
DATEDIED              95
L_THREAT               1
ER_VISIT               1
HOSPITAL               1
HOSPDAYS              33
X_STAY                 1
DISABLE                1
RECOVD                 3
VAX_DATE             278
ONSET_DATE           206
NUMDAYS              115
V_ADMINBY              9
V_FUNDBY               4
SPLTTYPE            5895
FORM_VERS              2
TODAYS_DATE          104
BIRTH_DEFECT           1
OFC_VISIT              1
ER_ED_VISIT            1
SYMPTOM1            2301
SYMPTOMVERSION1        2
SYMPTOM2            2220
SYMPTOMVERSION2        2
SYMPTOM3            2017
SYMPTOMVERSION3        2
SYMPTOM4            1833
SYMPTOMVERSION4        2
SYMPTOM5            1617
SYMPTOMVERSION5        2
